In [ ]:
#;.pykx.disableJupyter()

In [ ]:
# https://code.kx.com/pykx/3.0/examples/jupyter-integration.html#q-first-mode
import pykx as kx
kx.util.jupyter_qfirst_enable()

# Iterators
**Learning objectives**

To understand:
* What are iterators?
* Mapping iterators
* Accumulating iterators
* Where to learn more

# Iterators - Introduction

Iterators (previously known as adverbs) modify the behaviour of a **function** in the following way:
* Changes the function's application so that it is applied **iteratively**. 
* Replaces the use of loops as commonly seen in other programming languages 

# What are iterators?

##### Latent Iteration 

Before moving onto the specific iteration functions note that kdb+/q, in many cases, already acts in  a *vector* fashion, with 2 vectors:

In [ ]:
1 2 3+ 2 3 4 //natively pairwise

or with a scalar (atomic value) and a vector:

In [ ]:
2* 5 6 7 

No need for loops!

Cases with ambiguity will throw an error - e.g. if we try to add `2 3` to `4 5 6`:

In [ ]:
2 3 + 4 5 6

# Mapping iterators
Modify a function's application to iterate across every item in a list and are kdb+'s idea of a "for" loop. 

This iteration is explicit in our naming of these operators, all of which are known as some variant of "each": 

* each - (`each`)
* parallel each - (`peach`) 
* each-both - (`'`)
* each left - (`\:` note leaning left at top)
* each right - (`/:` note leaning right at top)
* each prior - (`':` or `prior`)


## each
The [`each`](https://code.kx.com/v2/ref/each/) iterator is used to apply a function to each item in a list - similar to "for x in list; do function(x)" in other languages.  

With a list of lists, we can use the operator count to tell us how many items we have in our list: 

In [ ]:
show L:(1 2 3;10 20i;30 40 50f;60)
count L   //we have 4 items in our list

To modify `count` so that it applies to each element in the list, use `each`: 

In [ ]:
count L
count each L //how many elements are in each list

We can apply different monadic functions to the **each** iterator. See some examples below: 

In [ ]:
type each L     // determining the type of each list
reverse each L  // reverse each list

The above examples use infix notation, i.e. `monadicFunc each arg_list`.

We can also use the bracket notation:

In [ ]:
each[avg;L]   //each modifies the behaviour of avg so that it is applied to each item of the list L 

`each` will not work with multivalent functions, only functions which take one argument. 

The keyword [`in`](https://code.kx.com/q/ref/in/) as an example takes two inputs: 

In [ ]:
3 in 1 2      //in takes two inputs
3 in 1 2 3 4  //infix notation 
in[3;1 2 3 5] //functional notation

In [ ]:
3 in each (1; 1 2 ; 3 4 )  //this doesn't work - how would you write this functionally?

We can project the function `in` to a single input function by fixing the first parameter:

In [ ]:
isThreeIn:3 in        // create a projection of in, making a monadic function that can now be used with each
isThreeIn

In [ ]:
isThreeIn 2 3 4 5

In [ ]:
isThreeIn each(1;1 4;2 3;3 5 6)

<img src="../qbies.png" width="50px" style="width: 50px;padding-right:5px;padding-top:15px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> Since <code>each</code> applies functions to every argument in a list, aggregate functions like <code>max</code> which we have seen return atomic values, will return <b>lists</b> of atomic values when used in conjunction with each. </i></p>


##### Exercise 
`L:(1 2 3;10 20i;30 40 50f;60)`
* Find the first element of L (no iterator required) 
* Find the first element of each item in L (indexing won't work for this - do you know why?)
* Find the min of each element of L

In [ ]:
show L:(1 2 3;10 20i;30 40 50f;60)

In [ ]:
//first element of L
first L 
first each L 

In [ ]:
// (indexing won't work for this - do you know why?)
L[;0]  //elide the index for each item 

In [ ]:
//this doesn't work because not each item in our list L is a list itself 
type each L  //the final item is an atom, and we can't index into an atomic value 
3f[0]

In [ ]:
//mininum element in each item of L
min each L

In [ ]:
//Write your code here


##### Exercise 

Using the keyword [`within`](https://code.kx.com/q/ref/within/) (and by creating a projection) test if `5` is within each of the following ranges: `(3 6; 4 8; 10 15)`

In [ ]:
within[5] each (3 6; 4 8; 10 15)

In [ ]:
//Write your code here 


## Parallel each (`':` or `peach`)

[Parallel each](https://code.kx.com/q/basics/peach/) - `peach` is the same as `each` but makes use of secondary threads if available. 

We can check if we have secondary threads with the following command: 

In [ ]:
//check secondary thread setting 
\s       

In [ ]:
\t:100 ({sum exp x?1.0}' )4#1000000  / each

In [ ]:
\t:100 ({sum exp x?1.0}':)4#1000000  / peach

<img src="../qbies.png" width="50px" style="width: 50px;padding-right:5px;padding-top:15px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> An important thing to know about parallel execution in kdb+/q is that secondary threads can't update the main process - we will get a <code>noupdate</code> error if our code tries to! </i></p>

## each-both

The dyadic iterator `'` (known as [each-both](https://code.kx.com/q/ref/maps/#each)), modifies a functions operation to use the items from two lists of the same length in a pairwise fashion (or using a list and an atom). 

This dyadic iterator can be applied to a dyadic function. Let's take `#` as an example:

In [ ]:
H:("the";"quick";"brown";"fox")   // list
3#H                               // take the first 3 items of the list

To see the pairwise nature of each-both we change the left-hand-side argument to be a list of equal length to our list `H`: 

In [ ]:
1 2 3 4#'H       //infix - pairing the operation # to be applied between corresponding values in H and (1 2 3 4) 
'[#][1 2 3 4;H]  //funcitonal notation - ' is operating on # to modify it's behaviour
#'[1 2 3 4;H]    //another funtional form (without being "funcitonal" about the arugment to ')

We can use the each-both iterator with an atom also:

In [ ]:
3#'H                         // "3 take each-both" - here the atomic value is extended, like with 2 + 1 2 3 
#[3] each H                  // equivalent - since we're not using pairwise nature


`3#'H` (three take each-both H) returns the first three items of each item.

<img src="../qbies.png" width="50px" style="width: 50px;padding-right:5px;padding-top:5px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> Each-both can only be applied to lists of equal length or when using a list with an atom.</i></p>

Returning to our example using `in` from before, we can avoid the projection by instead using `'` for this purpose.

In [ ]:
3 in'(1;1 4;2 3;3 5 6)
3 3 3 3 in'(1;1 4;2 3;3 5 6)  //equivalent

If we wanted to do this pairwise for other values we can extend to that: 

In [ ]:
3 1 2 4 in' (1;1 4;2 3;3 5 6)

##### Exercise
Join `("ni ff";1 2 3)` and `("o tsalB";4 5)` so that numbers are together and letters are together and assign this new list to a variable `spaceship`. Finally, reverse the order of each item of the list.

In [ ]:
//joining and assigning to new variable spaceship 
show spaceship: ("ni ff";1 2 3),'("o tsalB";4 5)

In [ ]:
//reversing each item 
show spaceship:reverse each spaceship

In [ ]:
//Write your code here

##  each-left (`\:`) and each-right (`/:`)

If we have two lists and we want to use one list as an individual argument and the other list as an iterative input we can use these two iterators.

The iterators [each-left](https://code.kx.com/q/ref/maps/#each-left-and-each-right) and [each-right](https://code.kx.com/q/ref/maps/#each-left-and-each-right), denoted by `\:` (backslash colon) and `/:` (forward slash colon) modify dyadic functions and operators. 

Each-left will modify a function to take the entire second argument as one input, and then apply the function iteratively to each item of the first argument. Similarly, each-right does the same but in the other direction.

**each-right**

Previously we tried to add `2 3` to `3 4 5` which threw a `'length` error. If we want to add `2 3` to each item of `3 4 5` we can do so using each-right (`/:`) to modify `+` as follows:

In [ ]:
2 3 +/: 3 4 5   //adding 2 3 to each item to the right

This is the same as individually doing:

In [ ]:
(2 3) + 3
(2 3) + 4
(2 3) + 5

Other examples:

In [ ]:
2 3 ,/: 3 4 5   //joining 2 3 to each item to the right
2 3 */: 3 4 5   //multiplying 2 3 by each item to the right

Applying `/:` returns a list with the same number of items as our RHS argument since that is what we have iterated over. In the case of `+` and `*` each sublist has the same number of items as our LHS. 

**each-left**

What do you expect to be different in our output and behaviour when we use `\:`? 

In [ ]:
2 3 +\: 3 4 5   //adding 3 4 5 to each item to the left
2 3 ,\: 3 4 5   //joining 3 4 5 to each item to the left
2 3 *\: 3 4 5   //multiplying 3 4 5 by each item to the left

Looking at the `in` operator again, we can extend our behaviour beyond our previous example and check for multiple values as follows:

In [ ]:
3 in'(1;1 4;2 3;3 5 6)         //previously 
3 4 in/:(1;1 4;2 3;3 5 6)      //extended to check for both 3 and 4  

Take time to work through what's really happening in the above and make sure you're comfortable with it!

**Common usage** 

Knowing what we now know about iterators, we can extend the functionality of `like` to compare our string with multiple patterns.  

In [ ]:
show L:("kdb+ is the fastest time-series database";"kdb+/q  has a lambda architecture";"q is the programming language") 
L like\: "*kdb+*" 

We might want to check for a number of patterns within a column. 

We can use `/:` to check if our column matches any of our patterns: 

In [ ]:
//defining orderIDs and patterns we are searching for 
ids: ("A123";"A234";"B123";"B234")
patterns:("A*";"*123") 

In [ ]:
//looking for our patterns 
patternCheck: ids like/: patterns    //we get booleans indicating for each id if they match the pattern
patternCheck

In [ ]:
//do we have any matches? Which IDs matched? 
any patternCheck                          //using "any" we can check if the ids match with any of the patterns
ids where any patternCheck                //putting it together using "where" we can retrieve the matching IDs!

<img src="../qbies.png" width="50px" style="width: 50px;padding-right:5px;padding-top:10px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i>Checking for multiple patterns in string columns within a table is a <b>very</b> common occurrence! The syntax for this is <code>any stringCol like/: patterns</code>.</i></p>

##### Exercise
Create a multiplication matrix using the list `0 1 2 3 4`. That is, a matrix whose (ij) entry is `i*j` which should look like:


    0  0  0  0  0
    0  1  2  3  4
    0  2  4  6  8
    0  3  6  9  12
    0  4  8  12 16

In [ ]:
row: til 5 
row*/:row    //multiply by each right
row*\:row    //multiplying by each left is the same in this case since both sides are the same i.e. row~row

In [ ]:
//write your code here

##### Bonus Code Golf Answer

Code golf is the like the real game of golf where scoring lower is better! If you want to challenge yourself the minimal character count for this is 11 - can you get it?

In [ ]:
i*\:i:til 5

## Each Prior (`':` or `prior`)
[Each prior](https://code.kx.com/q/ref/maps/#each-prior) `':` modifies a dyadic function to apply the function to each adjacent pair of items in a list.

 The `prior` iterator is [uniform](https://code.kx.com/q/basics/glossary/#uniform-function) on its argument, meaning that the output is the same length as the input.
 
Each prior can be called either using `':` or by the keyword `prior` and is best illustrated by example:

In [ ]:
0 +': 1 2 3
+':[1 2 3]         //same result as above but different syntax
prior[+;1 2 3]     //using prior - infix syntax (+) prior 1 2 3

In all the above return, our result is 1 (the first element) joined with (1+2) then (2+3). 

<img src="../qbies.png" width="50px" style="width: 50px;padding-right:5px;padding-top:5px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> It is good coding practice to use the <code>prior</code> keyword for clarity.</i></p> 

It is more common to use `-` with this iterator since in time-series analysis its usually more helpful to know how something has changed: 

In [ ]:
0 -': 1 2 3 60 62 62
-':[1 2 3 60 62 62] //same result as above but different syntax

<img src="../qbies.png" width="50px" style="width: 50px;padding-right:5px;padding-top:15px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> Some uses of each prior were so common that keywords were added to perform those operations - e.g. <code>deltas</code> which is actually <code>-':</code>.  It is good coding practice to use <code>deltas</code>. </i></p> 

In [ ]:
-':[200 300 100 500 400 200 100 -200 400 -100]
deltas 200 300 100 500 400 200 100 -200 400 -100 

##### Exercise 

* Create a function called `myMax` that will return the max of two inputs
* Use this function with each Prior (`':`) to return the pairwise rolling maximum across the list `(20 30 2 3 20 40 70)`

In [ ]:
myMax:{max x,y}
myMax':[20 30 2 3 20 40 70]

In [ ]:
//Write your code here

# Further Reading 
 
Iterators are a very interesting and advanced topic in kdb+/q - there is a good chance the more of them you use, the more efficient your code is.

**Accumulating iterators**

[Accumulation](https://code.kx.com/q/ref/accumulators/) iterators execute repeatedly over the values returned from calling the modified function. 

There are two accumulator iterations in kdb+/q, both of which operate in the exact same fashion, excepting their return value. These functions are: 
* Scan (`\`) 
* Over (`/`)

The difference between the two is that when scan (`\`) returns the intermediate values associated with each execution, while over (`/`) returns just the final value. Over requires less memory. 

These are *advanced iterators* and are covered in the **Advanced kdb+** course, though some small introduction to these is given in the Practical Guidance sheet. 

* The [Iterators](https://code.kx.com/q/wp/iterators/) whitepaper offers a thorough treatment and would be considered valuable additional reading.